# INSTALLS AND IMPORTS

In [ ]:
### INSTALL DEPENDENCIES
!pip install python-Levenshtein
!pip uninstall nltk
!pip install nltk==3.6.2

In [2]:
### IMPORT LIBRARIES
import pandas as pd
import numpy as np
import sys
import os
import Levenshtein as Lev

# FUNKTIONER

In [3]:
def wer(s1, s2):
    """
    Computes the Word Error Rate, defined as the edit distance between the
    two provided sentences after tokenizing to words.
    Arguments:
        s1 (string): space-separated sentence
        s2 (string): space-separated sentence
    """

    # build mapping of words to integers
    b = set(s1.split() + s2.split())
    word2char = dict(zip(b, range(len(b))))

    # map the words to a char array (Levenshtein packages only accepts
    # strings)
    w1 = [chr(word2char[w]) for w in s1.split()]
    w2 = [chr(word2char[w]) for w in s2.split()]

    return Lev.distance(''.join(w1), ''.join(w2))

def wer_normalized(s1, s2):
  return wer(s1.lower(),s2.lower()) / len(s2.split(" "))
def calculate_wer(df):
  return wer_normalized(df['corrected'],df['reference_text'])

# ByT5 evaluation



In [4]:
df = pd.read_pickle('/Users/jenspt/Desktop/git/DL-ByT5/data/ByT5_lr_1e4.pkl')

In [5]:
df['corrected_wer'] = df.apply(calculate_wer,axis=1)

In [7]:
print("MLN WER: ", df['corrected_wer'].mean()*100)


MLN WER:  6.465655469246048


# BLEU & GLEU

In [8]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.gleu_score import sentence_gleu

In [9]:
def calculate_bleu_baseline_normalized(df):
  total_bleu = 0
  for i in range(len(df)):
    ref = [df['reference_text'].iloc[i].split(" ")]
    hyp = df['transcription'].iloc[i].split(" ")
    sentence_bleu_score = sentence_bleu(ref, hyp)
    total_bleu += sentence_bleu_score
  return total_bleu / len(df)

def calculate_gleu_baseline_normalized(df):
  total_gleu = 0
  for i in range(len(df)):
    ref = [df['reference_text'].iloc[i].split(" ")]
    hyp = df['transcription'].iloc[i].split(" ")
    sentence_gleu_score = sentence_gleu(ref, hyp)
    total_gleu += sentence_gleu_score
  return total_gleu / len(df)

def calculate_bleu_normalized(df):
  total_bleu = 0
  for i in range(len(df)):
    ref = [df['reference_text'].iloc[i].split(" ")]
    hyp = df['corrected'].iloc[i].split(" ")
    sentence_bleu_score = sentence_bleu(ref, hyp)
    total_bleu += sentence_bleu_score
  return total_bleu / len(df)

def calculate_gleu_normalized(df):
  total_gleu = 0
  for i in range(len(df)):
    ref = [df['reference_text'].iloc[i].split(" ")]
    hyp = df['corrected'].iloc[i].split(" ")
    sentence_gleu_score = sentence_gleu(ref, hyp)
    total_gleu += sentence_gleu_score
  return total_gleu / len(df)


In [10]:

#corpus_bleu_score = calculate_bleu_corpus(df)
norm_bleu_score = calculate_bleu_normalized(df)
#calculated_gleu = calculate_gleu(df) #RuntimeError: generator raised StopIteration


#print("\nCorpus bleu:")
#print(corpus_bleu_score)
print("\nNormalized bleu:")
print(norm_bleu_score)
#print(calculated_gleu)

#corpus_bleu_score = calculate_bleu_corpus(df)
norm_gleu_score = calculate_gleu_normalized(df)
#calculated_gleu = calculate_gleu(df) #RuntimeError: generator raised StopIteration


#print("\nCorpus bleu:")
#print(corpus_bleu_score)
print("\nNormalized gleu:")
print(norm_gleu_score)
#print(calculated_gleu)

/opt/homebrew/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/homebrew/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/homebrew/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.


Normalized bleu:
0.8320419867936647

Normalized gleu:
0.8952613840069592


In [ ]:
## GLEU SCORES
# Baseline (transcription)= 0.8534873003494211
# ByT5_lr_1e4             = 0.8952613840069592
# ByT5_ws3000             = 0.8873560947109903
# ByT5                    = 0.887649053358017
# ByT5_extra_layer        = 0.0
# MultiLexNorm_raw_data   = 0.7097791013001765
# MultiLexNorm_base       = 0.6818061444602583
# MultiLexNorm_Fine_tuned = 0.1129387766622926


## BLEU SCORES (normalized)
# Baseline (transcription)= 0.7871290040412282
# ByT5                    = 0.8233903148477896
# ByT5_lr_1e4             = 0.8320419867936647
# ByT5_ws3000             = 0.8231682536463248
# ByT5_extra_layer        = 0.0
# MultiLexNorm_raw_data   = 0.6473379750494206
# MultiLexNorm_base       = 0.6190157431613983
# MultiLexNorm_fine_tuned = 0.006133337391333845
